# <u> <font color = "bisque"> Tareas:</font> </u>
## <font color = "bisque"> * Implementar procesos de renombre en TOPICO PESCAS</font> 

---

<b> <font color = "LightSalmon"> MODO DE TRABAJO PROCESO DE RENOMBRE - v2 </font> </b>

Voy a trabajar sobre argendata/data que esté hasta ahora, en un fork propio.

<b> Sobre argendata/data lo que voy a hacer es tomar los .csv  originales y hacer lo siguiente (dependiendo el caso), generando .csv actualizados:</b>


   - Si un dataset tiene un código que esté incluido en los geocodigos del geonomenclador, se nomencla con la string de name_long

   - Si un dataset tiene un código que NO este incluido en los geocodigos y tiene una string que matchee sin ambigüedad en desc_fundar, entonces se usa el código que está en el geonomenclador (y se guarda la asociación del codigo encontrado con el codigo usado)

   - Si un dataset tiene un código que NO este incluido en los geocodigos y tiene una string que NO matchea sin ambigüedad en desc_fundar, se genera un código nuevo (a revisión para ser desambiguado con posibles matches)

   - Si un dataset tiene una string que matchee sin ambigüedad en desc_fundar entonces se usa el código que está en el geonomenclador para esa string

   - Si un dataset tiene una string que NO matchee sin ambigüedad en desc_fundar, entonces se genera un código nuevo (a revisión para ser desambiguado con posibles matches)


<b> ¿Cómo busco las columnas de interés para esta iteración? </b>
- X% (por ejemplo 80%) del contenido de columnas de interés de un .csv de un tópico es parte de alguna de las columnas del geoneomenclador 
- Por el momento solo código que esté en el geonomenclador en español

Observaciones: 
- PARA ESTE LABURO ARMÉ MI PROPIO BLOQUE DE ENCODERs, DELIMITERs
- Para esta versión de la tarea, en principio, solo consideraría contenido en español

<b> <font color = "LightSalmon"> Productos: 
- .csv argendata con agregado de columnas para implementar renombres correspondientes
- .csv con lo que mencioné antes con .csv de argendata que presentarían problemas </font> </b>


---

###  <u> <font color = "orangered"> Cómo abrir cada .csv considerando su encoding: </font> </u>

<font color = "gold"> Cada .csv de Argendata tiene su propio encodings y delimiters </font>. 

No todos los .csv que contempla Argendata tienen el mismo enconding y delimiter. Por eso antes de comenzar a trabajar voy a armar el siguiente input para la tarea: dataframe que contenga 4 columnas que sean 'TOPICO', 'archivo_csv', 'encoding' y delimiter'.

In [1]:
import os
import glob
import chardet
import csv
import pandas as pd

# ruta de tabajo donde estan los topicos con sus csv 
base_dir = '/home/capuccino/Desktop/TrabajoFundar/pesca_renaming/data_argendata' # '/home/lucia/Desktop/Fundar/Argendata_renaming/data/data_argendata' 

# lista para recolectar los datos
data = []

# tamanio del fragmento a leer (en este caso 100000 bytes)
sample_size = 100000

for topico in os.listdir(base_dir): # ejemplo SEBACO
    topico_path = os.path.join(base_dir, topico) #ejemplo /home/lucia/Desktop/Fundar/Argendata_renaming/data/data_argendata/SEBACO

    # ignorar archivos sueltos como LICENSE y README.md
    if not os.path.isdir(topico_path):
        continue

    for archivo in os.listdir(topico_path):
        if archivo.endswith('.csv'):
            archivo_path = os.path.join(topico_path, archivo)

            # leer un fragmento del .csv para análisis
            with open(archivo_path, 'rb') as f:
                raw_sample = f.read(sample_size)

            # detectar encoding
            encoding_detected = 'utf-8'  # por defecto
            try:
                encoding_info = chardet.detect(raw_sample)
                if encoding_info and encoding_info['encoding']:
                    encoding_detected = encoding_info['encoding']
            except:
                pass

            # decodificar el fragmento
            try:
                sample = raw_sample.decode(encoding_detected, errors = 'replace')
            except:
                sample = raw_sample.decode('utf-8', errors = 'replace') 

            # detectar delimitador
            delimiter_detected = ','
            try:
                sniffer = csv.Sniffer()
                dialect = sniffer.sniff(sample)
                delimiter_detected = dialect.delimiter
            except:
                pass

            # agregar al resultado
            data.append({
                'TOPICO': topico,
                'archivo_csv': archivo,
                'encoding': encoding_detected,
                'delimiter': delimiter_detected
            })


df_resultado = pd.DataFrame(data)
df_codificacion = df_resultado

print(df_codificacion)

# guardar como csv el archivo csvTopicoArgendata_encoding_and_delimiters
# path_df_encoding_delimiters = '/home/lucia/Desktop/Fundar/Argendata_renaming/data/'
df_codificacion.to_csv('/home/capuccino/Desktop/TrabajoFundar/pesca_renaming/' + 'PESCA_encoding_delimiters.csv', index = False)


    TOPICO                                        archivo_csv    encoding  \
0   PESCAS        31_consumo_per_capita_pescado_anio_pais.csv       utf-8   
1   PESCAS  16_valor_cantidad_precio_exportacion_pesquero.csv       ascii   
2   PESCAS  01_produccion_pesquera_captura_y_acuicola_por_...       utf-8   
3   PESCAS                                22_pbg_pesquero.csv       utf-8   
4   PESCAS             04_desembarque_especie_ultimo_anio.csv       ascii   
5   PESCAS                      14_evolucion_pib_pesquero.csv       ascii   
6   PESCAS  07_acuicultura_produccion_por_pais_ultimo_anio...       utf-8   
7   PESCAS  18_valor_precio_cantidad_expo_por_especie_anio...       ascii   
8   PESCAS                          11_total_desembarques.csv       ascii   
9   PESCAS  09_composicion_consumo_carne_animal_por_tipo_y...       utf-8   
10  PESCAS         29_produccion_vs_consumo_pesca_arg_evo.csv       ascii   
11  PESCAS              21_acuicultura_produccion_arg_evo.csv       ascii   

###  <u> <font color = "orangered">  Tareas concretas del proceso de renaming </font> </u>

<b> <font color = "gold"> Sobre argendata/data lo que voy a hacer es tomar los .csv  originales y hacer lo siguiente (dependiendo el caso), generando .csv actualizados: </font> </b>

   - si un dataset tiene un código que esté incluido en los geocodigos del geonomenclador, se nomencla con la string de name_long (si es que existe), y si no es así se nomencla con la string de desc_fundar correspondiente

   - Si un dataset tiene un código que NO este incluido en los geocodigos y tiene una string que matchee sin ambigüedad en desc_fundar, entonces se usa el código que está en el geonomenclador (y se guarda la asociación del codigo encontrado con el codigo usado)

   - Si un dataset tiene un código que NO este incluido en los geocodigos y tiene una string que NO matchea sin ambigüedad en desc_fundar, se genera un código nuevo (a revisión para ser desambiguado con posibles matches)

   - Si un dataset tiene una string que matchee sin ambigüedad en desc_fundar entonces se usa el código que está en el geonomenclador para esa string

   - Si un dataset tiene una string que NO matchee sin ambigüedad en desc_fundar, entonces se genera un código nuevo (a revisión para ser desambiguado con posibles matches)

<b> <font color = "gold"> Para hacer esto, empecemos por: </font> </b>

1. abriría el archivo según su encoding y delimiter dado por df_codificacion 

2. detectaría si cada .csv de tiene una columna que se corresponda a la columna geocodigo del csv geonomenclador según la siguiente condición: fijarse si, al menos, el 80% de sus filas se encuentra contenida en la columna geocodigo del geonomenclador

3. detectaría si cada .csv de tiene una columna que se corresponda a la columna desc_fundar del csv geonomenclador según la siguiente condición: fijarse si, al menos, el 80% de sus filas se encuentra contenida en la columna desc_fundar del geonomenclador

4. esta información guardarla en un dataframe llamado columnscsv_Geocodigo_descFundar, con formato de 4 columnas que sean 'TOPICO', 'archivo_csv', 'columna_Geocodigo', 'columna_DescFundar'. Donde sea 'columna_Geocodigo' es el nombre de la columna de dicho cvs que cumple la condición 60% (80%, X%) geocodigo del geonomenclador y 'descFundar' es el nombre de la columna de dicho csv que cumple la condición 60% (80%, X%) desc_fundar del geonomenclador. 

A la hora de hacer la comparación del 60%:
- De la columna analizada la columna, eliminar nulos
- normalizar espacios, mayúsculas, tildes, para que casos como los siguientes no rompan el X%: "NEA " y "NEA" o "pampeana" y "Pampeana" o "Córdoba" y "Córdoba" se reconozcan como idénticos

<b> <font color = "bisque"> Esto resulta en un df con: TOPICO | archivo_csv | columna_Geocodigo | columna_DescFundar </font> </b>


In [2]:
import os
import pandas as pd
import unicodedata

# funcion para eliminar tildes
def sin_tildes(s):
    nkfd = unicodedata.normalize('NFKD', s)
    return ''.join(c for c in nkfd if not unicodedata.combining(c))

# rutas y dataframes necesarios

base_dir =  '/home/capuccino/Desktop/TrabajoFundar/pesca_renaming/data_argendata' 
df_geonomenclador = pd.read_csv('/home/capuccino/Desktop/TrabajoFundar/Argendata_renaming/data/input' + '/geonomenclador_LuloTest.csv')

# sets normalizados de las columnas de interes del geonomenclador: geocodigo y desc_fundar 
# estoy probndo usar set para practicar, si es necesario paso a lista
set_geocod = set(
    df_geonomenclador['geocodigo']
      .dropna().astype(str) #elimino nulos
      .str.strip() # quitar espacios al inicio/final
      .str.upper() # pasar todo a mayusculas
      .apply(sin_tildes) # sacar tildes
)
set_desc = set(
    df_geonomenclador['desc_fundar']
      .dropna().astype(str)
      .str.strip()
      .str.upper()
      .apply(sin_tildes)
)

results = []

# los topicos (directorios) ignorando los archivos LICENSE y README.md
topics = []
for d in os.listdir(base_dir):
    item_path = os.path.join(base_dir, d)
    
    if os.path.isdir(item_path): # verifico que sea un directorio
        
        if d != 'LICENSE' and d != 'README.md':
            topics.append(d)

for topico in topics:
    topic_dir = os.path.join(base_dir, topico)
    # listo los .csv de este tópico
    csv_files = [f for f in os.listdir(topic_dir) if f.endswith('.csv')]
    for archivo in csv_files:
        # uso el encoding y delimiter de df_codificacion
        row = df_codificacion[
            (df_codificacion['TOPICO'] == topico) &
            (df_codificacion['archivo_csv'] == archivo)
        ]
        if row.empty:
            print(f'No hay info de encoding/delimiter para {topico}/{archivo}') # quiero probar no estar pifiando
            continue
        encoding  = row.iloc[0]['encoding']
        delimiter = row.iloc[0]['delimiter']
        path_csv  = os.path.join(topic_dir, archivo)

        # leo csv con el encoding delimiter
        try:
            df_csv = pd.read_csv(path_csv, encoding = encoding, delimiter = delimiter)
        except Exception as e: 
            print(f'Error leyendo {path_csv}: {e}') # imprimo el error
            continue

        col_geocod = None
        col_desc   = None

        # analizar cada columna del .csv
        for col in df_csv.columns:
            serie = (
                df_csv[col]
                  .dropna().astype(str)
                  .str.strip()
                  .str.upper()
                  .apply(sin_tildes)
            )
            n = len(serie)
            if n == 0:
                continue

            match_geo  = serie.isin(set_geocod).sum() / n
            match_desc = serie.isin(set_desc).sum()   / n

            if match_geo  >= 0.6 and col_geocod is None:
                col_geocod = col
            if match_desc >= 0.6 and col_desc   is None:
                col_desc = col

        results.append({
            'TOPICO': topico,
            'archivo_csv': archivo,
            'columna_Geocodigo': col_geocod,
            'columna_DescFundar': col_desc
        })

# creo df de interes
columnscsv_Geocodigo_descFundar = pd.DataFrame(results)

columnscsv_Geocodigo_descFundar

,TOPICO,archivo_csv,columna_Geocodigo,columna_DescFundar
0,PESCAS,31_consumo_per_capita_pescado_anio_pais.csv,iso3,pais
1,PESCAS,16_valor_cantidad_precio_exportacion_pesquero.csv,None,None
2,PESCAS,01_produccion_pesquera_captura_y_acuicola_por_...,iso3,pais_nombre
3,PESCAS,22_pbg_pesquero.csv,None,provincia
4,PESCAS,04_desembarque_especie_ultimo_anio.csv,None,None
5,PESCAS,14_evolucion_pib_pesquero.csv,None,None
6,PESCAS,07_acuicultura_produccion_por_pais_ultimo_anio...,iso3,pais_nombre
7,PESCAS,18_valor_precio_cantidad_expo_por_especie_anio...,None,None
8,PESCAS,11_total_desembarques.csv,None,None
9,PESCAS,09_composicion_consumo_carne_animal_por_tipo_y...,iso3,pais


In [3]:
df_geonomenclador

,geocodigo,desc_fundar,name_long,name_short,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,ABW,Aruba,NaN,NaN,NaN,NaN,NaN,NaN
1,AFE,África Oriental y Meridional,NaN,NaN,NaN,NaN,NaN,NaN
2,AFG,Afganistán,NaN,NaN,NaN,NaN,NaN,NaN
3,AFW,África Occidental y Central,NaN,NaN,NaN,NaN,NaN,NaN
4,AGO,Angola,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1019,DESHUM_AHDI.WOF,Ramificaciones de Occidente (AHDI),Ramificaciones de Occidente,Ramificaciones de Occidente,NaN,NaN,NaN,NaN
1020,NIR,Irlanda del Norte,NaN,NaN,NaN,NaN,NaN,NaN
1021,CAMCLI_IA,Aviación internacional,NaN,NaN,NaN,NaN,NaN,NaN
1022,F_ESTPRO,África (OECD),África,África,NaN,NaN,NaN,NaN


<b> <font color = "gold"> Continuamos con lo que enumero a continuación para cumplir las condiciones detalladas enteriormente (los 5 bullets de más arriba): </font> </b>

Ya generamos el df columnscsv_Geocodigo_descFundar con: TOPICO | archivo_csv | columna_Geocodigo | columna_DescFundar, 
Donde 'columna_Geocodigo' es el nombre de la columna de dicho cvs que cumple la condición X% geocodigo del geonomenclador y 'descFundar' es el nombre de la columna de dicho csv que cumple la condición X% desc_fundar del geonomenclador.

<b> <font color = "gold">Por cada tópico (carpeta) y cada csv dentro de este, busco generar una nueva carpeta de cada tópico y dentro de cada carpeta guardar una copia de sus respectivos csvs pero agregandoles a los mismos dos columnas: una que sea 'geocodigoFundar' y otra que sea 'geonombreFundar'.</font> </b>

Si para un csv de un topico tanto columna_Geocodigo como columna_DescFundar es NA, ya sabremos que el csv copia se guardará con NA tanto en las columnas 'geocodigoFundar' como 'geonombreFundar'.

Siendo que columnscsv_Geocodigo_descFundar ya tiene por cada tópico y csv, si dicho csv tiene una columna 'columna_Geocodigo' que sería de geocodigo (condicion X% de columna geocodigo del df_geonomenclador) y otra columna 'columna_DescFundar' (condicion del 80 % de la columna desc_fundar del df_geonomenclador), voy a usarcolumnscsv_Geocodigo_descFundar para completar 'geocodigoFundar' y 'geonombreFundar'.

Para rellenar las columnas 'geocodigoFundar' y 'geonombreFundar', hay que tener las siguientes consideraciones:

- Si un dataset (cada csv original de cada topico) tiene una columna asociada a geocodigo segun columnscsv_Geocodigo_descFundar, el contenido de la columna geocodigoFundar del csv copia tendrá el contenido de la columna asociada a geocodigo segun columnscsv_Geocodigo_descFundar. La columna geonombreFundar del csv copia llenará cada una de sus filas acorde al name_long (si es que existe), y si no es así al string de desc_fundar correspondiente a ese geocodigo del geonomenclador.

- Si un dataset tiene una string que matchee sin ambigüedad en desc_fundar del geonomenclador entonces se usa el código que está en el geonomenclador para esa string. Similar al caso anterior.

- En el caso de que un dataset tiene un código que NO esté incluido en los geocodigos pero tiene una string que matchee sin ambigüedad en desc_fundar, entonces se usa el código que está en geocódigo en el geonomenclador. Esto se registrará en df_problemas con las columnas TOPICO | csv | problema: "dataset tiene el codigo [...], que NO esta incluido en los geocodigos pero tiene la string [...] que matchea sin ambiguedad en desc_fundar. No hay que modificar geonomenclador.". Por cada problema se guarda la string y el código correspondiente. Esto puede dar lugar que un mismo csv de un tópico tenga varios problemas a registrar. NO HAY QUE MODIFICAR GEONOMENCLADOR, EL RENAMING ES HECHO A PARTIR DE LA COLUMNA desc_fundar DEL GEONOMENCLADOR. 

- Si un dataset tiene un código que NO este incluido en los geocodigos y tiene una string que NO matchea sin ambigüedad en desc_fundar, se genera un código nuevo. Esto se registrará en df_problemas con las columnas TOPICO | csv | problema: "dataset tiene el codigo [...] que NO esta incluido en los geocodigos y tiene la string [...] que NO matchea sin ambiguedad en desc_fundar. Puede implicar modificaciones del geonomenclador (a revisión, para no pisarse con otros casos).". Por cada problema se guarda la string y el código correspondiente. Esto puede dar lugar que un mismo csv de un tópico tenga varios problemas a registrar. PODRÍA DAR LUGAR A MODIFICACIONES DEL GEONOMENCLADOR. 

    -Situacion similar a esta: Dataset tiene el código [...] que NO está incluido en los geocódigos y no existe columna desc_fundar. En df_problemas se reporta: "dataset tiene el codigo [...] que NO esta incluido en los geocodigos y no existe columna desc_fundar. Puede implicar modificaciones del geonomenclador (a revisión, para no pisarse con otros casos)."

- Si un dataset tiene una string que NO matchee sin ambigüedad en desc_fundar, entonces guardar este caso en el dataframe llamado df_problemas con las columnas TOPICO | csv | problema, donde TOPICO es el topico con el que se corresponde, csv el csv evaluado y en problema poner: "Para la string [...] se debería generar un codigo nuevo (a revisión, para no pisarse con otros casos)". PODRÍA DAR LUGAR A MODIFICACIONES DEL GEONOMENCLADOR. 

In [4]:
import os
import pandas as pd
import unicodedata

# funcion eliminar tildes
def sin_tildes(s):
    nkfd = unicodedata.normalize('NFKD', s)
    return "".join(c for c in nkfd if not unicodedata.combining(c))

# rutas
base_dir = '/home/capuccino/Desktop/TrabajoFundar/pesca_renaming/data_argendata'
dest_dir = '/home/capuccino/Desktop/TrabajoFundar/pesca_renaming/data_argendata_new'

# df inputs
df_cod = df_codificacion
df_geo = df_geonomenclador
df_cols = columnscsv_Geocodigo_descFundar  

# dicc de name_long y desc_fundar por geocodigo del geonomenclador argendata
geonames_by_geocode = {}
for _, row in df_geonomenclador.iterrows():
    raw_code   = str(row['geocodigo'])
    code_norm  = sin_tildes(raw_code.strip().upper())
    name_long  = row['name_long']
    desc_fund  = row['desc_fundar']
    geonames_by_geocode[code_norm] = (name_long, desc_fund)

# lista problemas por tipo
problemas = []

# lista de topicos (en este caso es solo pesca)
topics = []
for d in os.listdir(base_dir):
    path = os.path.join(base_dir, d)
    if os.path.isdir(path) and d not in ('LICENSE', 'README.md'):
        topics.append(d)

# proceso cada topico y csv
for topico in topics:
    src_topic = os.path.join(base_dir, topico)
    dst_topic = os.path.join(dest_dir, topico)
    os.makedirs(dst_topic, exist_ok = True)
    
    # listar los archivos csv en este tópico
    for archivo in os.listdir(src_topic):
        if not archivo.lower().endswith('.csv'):
            continue
        
        # abro el csv con respectivos encoding y delimiter, cfg de configuracion 
        cfg = df_codificacion[
            (df_codificacion['TOPICO'] == topico) &
            (df_codificacion['archivo_csv'] == archivo)
        ]
        if cfg.empty:
            print(f"No hay info encoding/delimiter para {topico}/{archivo}")
            continue
        encoding = cfg.iloc[0]['encoding']
        delimiter = cfg.iloc[0]['delimiter']
        
        # del df columnscsv_Geocodigo_descFundar toma la info del csv y topico que me interesa
        row_cols = columnscsv_Geocodigo_descFundar[
            (columnscsv_Geocodigo_descFundar['TOPICO'] == topico) &
            (columnscsv_Geocodigo_descFundar['archivo_csv'] == archivo)
        ]
        col_code = row_cols.iloc[0]['columna_Geocodigo'] if not row_cols.empty else None
        col_desc = row_cols.iloc[0]['columna_DescFundar'] if not row_cols.empty else None
        
        # leo csv
        src_path = os.path.join(src_topic, archivo)
        df = pd.read_csv(src_path, encoding = encoding, delimiter = delimiter)
        
        # le sumo al csv las nuevas columnas de interes
        df['geocodigoFundar'] = pd.NA
        df['geonombreFundar'] = pd.NA
        
        # recorrer filas del csv
        for idx in df.index:

            # CASO A: existe columna correspondiente a geocodigo del geonomenclador en el csv

            # A.0) hay un codigo presente en geocodigos, lo relleno con name_long o desc_fundar
            
            if pd.notna(col_code):
                raw_code = str(df.at[idx, col_code])
                norm_code = sin_tildes(raw_code.strip().upper())
                if norm_code in geonames_by_geocode:
                    nl, desc0 = geonames_by_geocode[norm_code]
                    df.at[idx, 'geocodigoFundar'] = norm_code
                    df.at[idx, 'geonombreFundar'] = nl if pd.notna(nl) else desc0 # relleno con name_long si existe, sino relleno con la sting de desc_fundar
                else:
                    # A.1) hay un codigo NO presente en geocodigos, pero intento por columna desc_fundar
                    if pd.notna(col_desc):
                        raw_desc = str(df.at[idx, col_desc])
                        norm_desc = sin_tildes(raw_desc.strip().upper()) # normalizo columna del csv de interes que se corresponderia a desc_fundar segun condicion del X%
                        # bsqueda de coincidencias en desc_fundar
                        matched_code = []
                        for code_key, (_nl, desc0) in geonames_by_geocode.items(): # _nl para hacer explicito que no uso nl en este caso
                            norm_desc0 = sin_tildes(str(desc0).strip().upper()) # normalizo desc_fundar del geonomenclador 
                            #(para comparacion con la columna del csv de interes columna del csv de interes que se corresponderia a desc_fundar segun condicion del X%)
                            if norm_desc0 == norm_desc:
                                matched_code.append(code_key)
                        if len(matched_code) == 1:
                            c0 = matched_code[0]
                            nl, desc0 = geonames_by_geocode[c0]
                            df.at[idx, 'geocodigoFundar'] = c0
                            df.at[idx, 'geonombreFundar'] = nl if pd.notna(nl) else desc0
                            problemas.append({
                                'TOPICO': topico,
                                'csv': archivo,
                                'problema': f"dataset tiene el codigo '{norm_code}', que NO esta incluido en los geocodigos pero tiene la string '{norm_desc}' que matchea sin ambiguedad en desc_fundar. No hay que modificar geonomenclador."
                            })
                        else:
                            problemas.append({
                                'TOPICO': topico,
                                'csv': archivo,
                                'problema': f"dataset tiene el codigo '{norm_code}' que NO esta incluido en los geocodigos y tiene la string '{norm_desc}' que NO matchea sin ambiguedad en desc_fundar. Puede implicar modificaciones del geonomenclador (a revisión, para no pisarse con otros casos)."
                            })
                    # A.2) no hay columna desc_fundar para intentar

                    else:
                        problemas.append({
                            'TOPICO': topico,
                            'csv': archivo,
                            'problema': f"dataset tiene el codigo '{norm_code}' que NO esta incluido en los geocodigos y no existe columna desc_fundar. Puede implicar modificaciones del geonomenclador (a revisión, para no pisarse con otros casos)."
                        })
            
            # CASO B: no existe columna de geocodigo, pero si se tiene de desc_fundar

            elif pd.notna(col_desc):
                raw_desc = str(df.at[idx, col_desc])
                norm_desc = sin_tildes(raw_desc.strip().upper())
                matched_code = []
                for code_key, (_nl, desc0) in geonames_by_geocode.items():
                    if sin_tildes(str(desc0).strip().upper()) == norm_desc:
                        matched_code.append(code_key)
                if len(matched_code) == 1:
                    c0 = matched_code[0]
                    nl, desc0 = geonames_by_geocode[c0]
                    df.at[idx, 'geocodigoFundar'] = c0
                    df.at[idx, 'geonombreFundar'] = nl if pd.notna(nl) else desc0
                else:
                    problemas.append({
                        'TOPICO': topico,
                        'csv': archivo,
                        'problema': f"Para la string '{norm_desc}' se debería generar un codigo nuevo. Puede implicar modificaciones del geonomenclador (a revisión, para no pisarse con otros casos)."
                    })
            
            # CASO C: si no hay columnas asociadas, quedan NaN
        
        # guardar csv copia modificado en su correspondiente topico, con UTF-8 y separador coma
        dst_path = os.path.join(dest_dir, topico, archivo)
        df.to_csv(dst_path, index = False,  encoding = 'utf-8', sep = ',')

# df de problemas
df_problemas = pd.DataFrame(problemas)
df_problemas.to_csv('/home/capuccino/Desktop/TrabajoFundar/pesca_renaming/df_problemas.csv', index = False,  encoding = 'utf-8', sep = ',')
print(df_problemas)



     TOPICO                                              csv  \
0    PESCAS      31_consumo_per_capita_pescado_anio_pais.csv   
1    PESCAS      31_consumo_per_capita_pescado_anio_pais.csv   
2    PESCAS      31_consumo_per_capita_pescado_anio_pais.csv   
3    PESCAS      31_consumo_per_capita_pescado_anio_pais.csv   
4    PESCAS      31_consumo_per_capita_pescado_anio_pais.csv   
..      ...                                              ...   
248  PESCAS  23_share_acuicola_total_pesca_por_pais_anio.csv   
249  PESCAS  23_share_acuicola_total_pesca_por_pais_anio.csv   
250  PESCAS  23_share_acuicola_total_pesca_por_pais_anio.csv   
251  PESCAS  23_share_acuicola_total_pesca_por_pais_anio.csv   
252  PESCAS  23_share_acuicola_total_pesca_por_pais_anio.csv   

                                              problema  
0    dataset tiene el codigo 'F5503', que NO esta i...  
1    dataset tiene el codigo 'F5503', que NO esta i...  
2    dataset tiene el codigo 'F5503', que NO esta i...  
3  